## Jupyter Notebook to Aid Development

In [11]:
import requests
import pandas as pd

url = "https://v1.data.uccf.io/api/christian-unions/expand"
response = requests.get(url)
data = response.json()
datadf = pd.DataFrame(data)

In [14]:
# institution is compiled here:
# institutions = datadf['institutions']
# manual appending of data so that institutions is a list of dicts
institutions = []

for i in range(len(datadf)):
    print(i)
    institutions.append(datadf['institutions'][i][0])
    institutions[i]["full_name"] = datadf['full_name'][i]
    # check if the social links exist
    for socials in ['website', 'twitter', 'facebook', 'instagram']:
        if datadf[socials][i] != None:
            # if exists, add to the insitutions dict
            institutions[i][socials] = datadf[socials][i]

print(institutions[0])
print(institutions[0]['full_name'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
{'id': 1, 'name': 'Aberdeen, University of', 'postcode': 'AB24 3FX', 'geocode': '57.165019900, -2.099122300', 'region': {'id': 1, 'name': 'Scotland'}, 'full_name': 'Aberdeen University Christian Union', 'website': 'https://linktr.ee/abdncu', 'facebook': 'https://www.facebook.com/abdncu/', 'instagram': 'https://www.instagram.com/abdncu'}
Aberdeen Univer

In [15]:
# second run through to get the click function to output url data
import dash
from dash import dcc, html, Output, Input
import plotly.graph_objects as go

# Parse geocodes
lats = []
lons = []
names = []
full_names = []

for i in range(len(institutions)):
    if institutions[i]['geocode'] is not None:
        lat, lon = (institutions[i]['geocode'].split(','))
        lats.append(float(lat))
        lons.append(float(lon))
    else:
        lats.append(0)
        lons.append(0)
    names.append(institutions[i]['name'])

# Create the Dash app
app = dash.Dash(__name__)

# Create the map figure
fig = go.Figure(go.Scattermapbox(
    lat=lats,
    lon=lons,
    mode='markers',
    marker=dict(size=12),
    text=names,
    hoverinfo='text'
))

fig.update_layout(
    mapbox=dict(
        style='open-street-map',  # Free, no token needed
        center=dict(lat=sum(lats)/len(lats), lon=sum(lons)/len(lons)),
        zoom=5
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=600,
    clickmode='event+select'  # Enable click events
)

app.layout = html.Div([
    html.H1('UK Institutions Map'),
    dcc.Graph(id='map', figure=fig),
    html.Div(id='institution-info', style={
        'padding': '20px',
        'marginTop': '20px',
        'border': '1px solid #ddd',
        'borderRadius': '5px'
    })
])

@app.callback(
    Output('institution-info', 'children'),
    Input('map', 'clickData')
)

def display_click_data(clickData):
    if clickData is None:
        return html.P('Click on a marker to see institution details and links')
    
    # Get the clicked institution index position
    point_index = clickData['points'][0]['pointIndex']
    inst = institutions[point_index]
    
    # Create the links display
    links = [
        html.H3(inst['name']),
        html.H3(inst['full_name']),
        html.P(f"Postcode: {inst['postcode']}"),
        html.P(f"Region: {inst['region']['name']}"),
        html.Hr(),
        html.H4('Links:'),

    ]

    # Define all possible social links with their icons and labels
    social_config = {
        'website': {'icon': '🌐', 'label': 'Official Website'},
        'twitter': {'icon': '🐦', 'label': 'Twitter'},
        'instagram': {'icon': '📷', 'label': 'Instagram'},
        'facebook': {'icon': '📘', 'label': 'Facebook'},
    }

    # logic to only show existing links
    found_links = False

    # logic to add only existing links
    for key, config in social_config.items():
        if key in inst:
            links.append(
                html.A(f"{config['icon']} {config['label']}", 
                       href=inst[key], 
                       target='_blank',
                       style={'display': 'block', 'margin': '10px 0', 'fontSize': '16px'})
            )
            found_links = True

    if not found_links:
        links.append(html.P("No links available for this institution."))
            
    
    return html.Div(links)

if __name__ == '__main__':
    app.run_server(debug=True, port=8051, use_reloader=False)